In [1]:
import sklearn
from sklearn import datasets
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np
import time

In [2]:
diabetes = datasets.load_diabetes()
data_X = diabetes.data
data_Y = diabetes.target

print(f"X Shape: {data_X.shape}")
print(f"Y Shape: {data_Y.shape}")

X Shape: (442, 10)
Y Shape: (442,)


In [3]:
data_X_train, data_X_test, data_Y_train, data_Y_test = train_test_split(data_X, data_Y, test_size=0.20)

In [4]:
mlp_reg = MLPRegressor(hidden_layer_sizes=(20, 10, 10, 10, 5), max_iter=int(1e9))
mlp_reg.fit(data_X_train, data_Y_train)
score = mlp_reg.score(data_X_test, data_Y_test)

print(f"R^2 Score: {score}")

R^2 Score: 0.38033503510786315


In [5]:
def generate_shift_matrix(shift, size, delta=1.0):
    # Construct the shift matrix.
    # Set entries with i = j + shift to 1.0.
    # Set entries with i != j + shift to 0.0.
    matrix = np.zeros((size, size))
    for k in range(size):
        index = shift - offset + k
        if index >= 0 and index < size:
            matrix[k, index] = 1
    return matrix

def generate_diff_matrix(shifts, size, delta=1.0):
    # Construct Taylor Series coefficient matrix for shift operators.
    # Element (i,j) should contain coefficients for the i-th derivative in the Taylor Series of the j-th shift.
    # We scale everything by (# of shifts - 1)! for numerical stability.
    taylor_size = len(shifts)
    taylor_matrix = np.empty((taylor_size, taylor_size))
    factorial = 1
    for i in range(taylor_size - 1, -1, -1):
        for j in range(taylor_size):
            taylor_matrix[i, j] = factorial * (shifts[j] ** i)
        if i > 0:
            factorial *= i

    # Construct desired coefficient vector for the resulting Taylor Series.
    # Every element should be zero except for the last since all derivatives should cancel.
    taylor_vector = np.zeros(taylor_size)
    taylor_vector[-1] = factorial

    # Solve the matrix to get the coefficients on the shift operators.
    coeffs = np.linalg.solve(taylor_matrix, taylor_vector)

    # Construct the difference matrix.
    # This is a linear combination of the shift matrices with the found coefficients.
    matrix = np.zeros((size, size))
    for k, shift in enumerate(shifts):
        matrix = matrix + coeffs[k] * generate_shift_matrix(shift, size)
    return matrix / delta ** (taylor_size - 1)

In [15]:
from findiff import FinDiff

delta = 0.01
degree = 4
degree_shape = (degree,) * data_X.shape[1]
n = points_necessary = int(np.ceil(degree / 2))

coeffs = np.zeros(degree ** data_X.shape[1])

domain = np.stack(np.meshgrid(*[np.linspace(-n * delta, n * delta, 2 * n + 1) for _ in range(data_X.shape[1])]), axis=-1)
domain_shaped = domain.reshape(-1, data_X.shape[1])
values_shaped = mlp_reg.predict(domain_shaped)
values = values_shaped.reshape(*domain.shape[:-1])

for d, index in enumerate(np.ndindex(degree_shape)):
    df = FinDiff(*((k, delta, idx) for k, idx in enumerate(index)), acc=1)
    coeffs[d] = df(values)[(n,) * data_X.shape[1]]

IndexError: Shift slice out of bounds

In [7]:
start = time.time()
predicted_Y = mlp_reg.predict(sample_values)
beta = np.linalg.lstsq(A, predicted_Y)
end = time.time()
print(f"Time Elapsed {end - start}s")

NameError: name 'sample_values' is not defined

In [ ]:
def compute_value(x):
    s = 0.0
    for d, index in enumerate(np.ndindex(*coeffs_shape)):
        s += np.prod(np.power(x, index))
    return s

In [ ]:
y_nn_predicted = mlp_reg.predict(data_X_test)
y_poly_predicted = np.zeros_like(data_Y_test)
start = time.time()
for k in range(len(y_poly_predicted)):
    y_poly_predicted[k] = compute_value(data_X_test[k])
    end = time.time()
    print(f"Progress ({k+1}/{len(y_poly_predicted)}) - Time Elapsed {end - start}s")

In [ ]:
print(f"R^2 Score NN: {r2_score(data_Y_test, y_nn_predicted)}")
print(f"R^2 Score Poly: {r2_score(data_Y_test, y_poly_predicted)}")